# CONSTRUCCIÓN DE LOS "FOOTPRINTS"

Construimos los footprints temporales de cada cliente a partir de sus transacciones, de tal manera que agrupamos en un intervalo de d días.


Dado:

    Sesion de TX
        s = {cliente, timestamp, Monto}
    
    Cada cliente tiene una secuencia de sesiones de TX (S)
        S = { s1, s2 , s3, ..., sn}                n: # de sesiones de cada cliente
        
        

In [1]:
# LIBRERIAS
import numpy as np
import datetime
from datetime import date
import json
import pylab
import pandas as pd
import matplotlib.pyplot as plt
import os, sys

## Preparacion de datos

### Cargando datos

In [2]:
def leer_data():
    infile='../RECURSOS/data.csv'
    data = pd.read_csv(infile)
    return data
data_original = leer_data()

### Preparacion de datos

In [3]:
data = data_original[['client_id','date','año','mes','dia','hora','mccg','quantity','amount_sol']]
data.head(5)

,client_id,date,año,mes,dia,hora,mccg,quantity,amount_sol
0,NNeQwQy9MAQ=,2016-07-15 22:23:25,2016,7,15,22,2,1,100.0
1,qFRoPHPOc/I=,2016-06-21 21:30:55,2016,6,21,21,2,1,20.0
2,qFRoPHPOc/I=,2017-02-01 01:29:59,2017,2,1,1,2,1,100.0
3,qFRoPHPOc/I=,2017-03-09 02:26:05,2017,3,9,2,2,1,50.0
4,he9pUcExpTY=,2016-10-01 06:45:48,2016,10,1,6,2,1,100.0


## Definicion de variables

### mccgs

In [4]:
# DEFINIMOS MCCGS: PARA CREAR UN FOOTPRINT DINAMICO
mccgs = ((data.groupby(['mccg'], as_index=False, sort=True)).count())['mccg']

### Clientes

In [5]:
# DEFINIMOS LA LISTA DE CLIENTES
clientes =  data.groupby('client_id').client_id.count().index
clientes

Index(['+9yJoBsAES4=', '+EUZmfQX5bA=', '+Ilmh+mr2WM=', '+JHjmW4a+e0=',
       '+JcXo4zlmNQ=', '+MD+jHUjjBs=', '+PREZ3ccYyY=', '+QSSr10KkNY=',
       '+QhRFlN5pIw=', '+T3IfxTS2u8=',
       ...
       'zLtR0lNrRys=', 'zMpGZsMS3Eo=', 'zQaVDdAnvBg=', 'zSl+zNri4Rg=',
       'zUJmWhTFVKE=', 'zgXGr0wAaqk=', 'zid9Qi0vD84=', 'zk41R+DpP70=',
       'zs3qMAU6HN8=', 'zs9yIYFt4EI='],
      dtype='object', name='client_id', length=4983)

# FOOTPRINT PARA CADA MCCG

## Unidad de TXs temporales (U)

Resume un conjunto de TXs en un periodo de tiempo **“tau”**.
<div>Los footprints que creamos será la agrupación de todas las transacciones en "tau" de cada cliente, y representará una unidad de comportamiento con 3 dimensiones.</div> 


Donde:
1.  **tau** = 1 semana
2.  dimensión 1:    **d**   = 7 días
3.  dimensión 2:	**t**   = 4 turnos	 
4.  dimensión 3:    **m**   = # de mccgs en nuestra data.

    
Para cada cliente:
    
    Entrada:   S = {s1, ..., sn}
    Salida:    SS = {U(1), U(2), U(3), ...,U(m)}     donde m <=n

### Funciones de apoyo

In [6]:
# definimos los 4 time_windows(t) que usaremos
def time_window(hora):
    tw = -1
    if hora >=0:
        tw = 0      # Madrugada
    if hora >=6:
        tw = 1      # Mañana
    if hora >=12:
        tw = 2      # Tarde
    if hora >=18:
        tw = 3      # Noche
    return tw

Definimos funcion para generar los footprint (U) de un usuario, con los parametros:

1. <div> **user**:  Dataset original filtrado para un cliente determinado</div> 
2. <div> **monto**: Indica si los footprint seran generados con la suma de los el numero de TXs (False) o la suma de los montos por cada TX (True)</div> 

In [7]:
def procesar_u(user, monto = False):    
    uid=list(user['client_id'])[0]              # Cliente_id
    years = set(list(user['año']))              # Lista los años en que se tiene TXs registradas
    anni = {year:{} for year in list(years)}    # definimos anni como una lista 
    
    # para cada fila, es decir, cada TXs del cliente)
    for dat in  range(0,len(user)):
        año = user.iloc[dat]['año']
        week=str(datetime.datetime(año,user.iloc[dat]['mes'],user.iloc[dat]['dia']).isocalendar()[1])
        if len(week)==1:
            week = '0'+week
        weekday=datetime.datetime(año,user.iloc[dat]['mes'],user.iloc[dat]['dia']).weekday()
        turn = time_window(user.iloc[dat]['hora'])
        mccg = user.iloc[dat]['mccg']
        
        # Si la semana no existe en el año
        if not(week in anni[año]):
            anni[año][week] = {}
        # Si el mccg no existe en la semana y año
        if not (mccg in anni[año][week]):
            anni[año][week][mccg]={}  #NUMERO DE MCCGs VARIABLES
        # Si el turno no existe en el mccg,semana y año
        if not (turn in anni[año][week][mccg]):
            anni[año][week][mccg][turn]=np.array([0]*7)  #CUATRO TURNOS
            
        if monto:
            # suma montos "importancia por gastos"
            anni[año][week][turn][weekday]+=user.iloc[dat]['amount_sol']
        else:
            # suma cantidades "importancia por compras"
            anni[año][week][mccg][turn][weekday]+=user.iloc[dat]['quantity'] 
            
    return uid,anni

### Procesando U

Generamos en formato json el footprint de cada cliente
donde el los índices son: **cliente_id**, **año**, **semana**, **mccg**, **turno** conteniendo un **array[0,...,6]** con los dias.



In [8]:
##################################################
#        Procesando U de cada CLIENTE
##################################################
    
profiles={}           # Creamos lista de prefiles
contador=0 
print("Number of rows "+str(len(data))) 

# Para cada cliente
for cliente in clientes:
    cliente_i= data[data['client_id'] == cliente]       # filtramos dataset solo para el cliente i
    results=procesar_u(cliente_i, monto=False)          # procesamos u del usuario i
    profiles[results[0]]=results[1]                     # cargamos lista en indice "uid" con la data del cliente(json)


Number of rows 10000


In [9]:
profiles

{'+9yJoBsAES4=': {2016: {'24': {3: {2: array([1, 0, 0, 0, 0, 0, 0])}}}},
 '+EUZmfQX5bA=': {2017: {'10': {2: {3: array([0, 0, 0, 0, 0, 0, 1])}}}},
 '+Ilmh+mr2WM=': {2017: {'19': {2: {3: array([0, 0, 0, 0, 0, 2, 0])}},
   '21': {2: {0: array([0, 0, 1, 0, 0, 0, 0])}},
   '01': {2: {0: array([0, 0, 0, 0, 0, 1, 0])}},
   '03': {2: {3: array([0, 0, 0, 0, 0, 0, 1])}},
   '06': {2: {2: array([0, 0, 0, 0, 0, 0, 1]),
     3: array([0, 0, 0, 0, 0, 0, 1])}}}},
 '+JHjmW4a+e0=': {2016: {'51': {2: {0: array([3, 0, 1, 0, 1, 0, 0]),
     3: array([1, 0, 0, 0, 0, 0, 0])}},
   '34': {2: {2: array([1, 0, 0, 0, 0, 0, 0])}},
   '30': {2: {0: array([0, 0, 0, 0, 0, 1, 0])}},
   '50': {2: {0: array([0, 0, 0, 0, 1, 0, 0])}},
   '40': {2: {2: array([0, 0, 1, 0, 0, 0, 0])}}},
  2017: {'01': {2: {0: array([0, 0, 1, 1, 0, 0, 0])}}}},
 '+JcXo4zlmNQ=': {2017: {'21': {2: {2: array([0, 0, 0, 0, 1, 0, 0])}},
   '12': {2: {3: array([0, 0, 0, 1, 0, 0, 0])}}}},
 '+MD+jHUjjBs=': {2017: {'19': {3: {2: array([0, 0, 1, 0, 0, 0

Creamos la cabecera dinámica donde se guardaran todos los footprints generados

In [10]:
cabecera = 'customer_id,year,week,profile_id,mccg,turn,size'
for i in range(len(mccgs)):
    for j in range(4):                # numero de turnos
        for k in range(7):            # numero de dias
            cabecera = cabecera+','+'m'+str(mccgs[i])+'t'+str(j)+'d'+str(k)
cabecera = cabecera+'\n'

In [11]:
outfile='./RESULTADOS/U'           # Indicamos archivo de salida 
individual_footprint="%s.footprint" %(outfile)
fw=open(individual_footprint,'w')  

fw.write(cabecera)                    # Escribimos la cabecera
footprints=0 

# Guardamos el Json como un dataframe
for uid in profiles:   #para cada uid (cliente)
    profile_id=0
    for year in profiles[uid]:                                    #######################
        for week in profiles[uid][year]:                          #######################
            temp=np.zeros(4*7*len(mccgs))
            for mccg in profiles[uid][year][week]:                #######################
                pos_mccg = list(mccgs).index(mccg) # posicion del mccg
                temp2 = np.zeros(4*7)
                for turn in profiles[uid][year][week][mccg]:      #######################
                    pos_turn = turn  # posicion del turno
                    d=profiles[uid][year][week][mccg][turn]       #######################        
                    
                    for i in range(pos_turn*7,(pos_turn+1)*7):
                        temp2[i] += d[i-(pos_turn*7)]
                
                for i in range(pos_mccg*28,(pos_mccg+1)*28):
                    temp[i] = temp2[i-(pos_mccg*28)]
                
                txt = ''+str(uid)+','+str(year)+','+str(week)+','+str(profile_id)+','+str(mccg)+','+str(turn)+','+str(sum(temp))
                for i in range(len(temp)):
                    txt = txt +','+str(temp[i])
                
                fw.write(txt +'\n')

            profile_id = profile_id + 1    # perfil cambia cada unidad de fecha diferente (eg. cada semana != es un perfil) de cada cliente
    footprints+=profile_id
    fw.flush()
fw.close()
print ("number of footprint: "+str(footprints))

number of footprint: 8020
